# Part 2: Phrase-structure parsing with CKY

In this part of the assignment, we'll train a simple phrase-structure (constituency) parser on the [Penn Treebank](https://catalog.ldc.upenn.edu/ldc99t42), then implement exact inference using the CKY algorithm.

You may want to refer to the Week 10 async material and lecture notes for a detailed discussion of the algorithm. The following may also be useful:
- [Guide to Penn Treebank constituent tags](http://www.surdeanu.info/mihai/teaching/ista555-fall13/readings/PennTreebankConstituents.html) - explains what `JJ`, `NNP`, and all the other tags mean.
- [Syntactic Parsing (Jurafsky & Martin, Ch. 12)](https://web.stanford.edu/~jurafsky/slp3/12.pdf) - particularly 12.1 and 12.2.
- [Accurate Unlexicalized Parsing](http://ilpubs.stanford.edu:8091/~klein/unlexicalized-parsing.pdf) (Klein & Manning, 2003)

We'll write code in three parts:
- **(a)** Initial preprocessing of the treebank
- **(b)** Calculation of production rule probabilities
- **(c)** Implementation of CKY algorithm

We provide the code for 1 and much of the framework surrounding 2 and 3.

In [1]:
# Install a few python packages using pip
from w266_common import utils
utils.require_package("tqdm")      # for progress bars
utils.require_package("graphviz")  # for rendering trees

### Preliminaries: GraphViz

**You can skip this if you have GraphViz set up from working on A2***

As with Assignment 2, this notebook uses [GraphViz](https://www.graphviz.org/) to render tree structures. On Ubuntu / Debian (including Google Cloud), you can install it by running on the command line:
```
sudo apt-get install graphviz
```

For Mac OSX, you can install using Homebrew:
```
brew install graphviz
```
or see https://www.graphviz.org/download/ for more options. Run the cell below to set up rendering and show a sample tree.

In [2]:
import nltk
from w266_common import treeviz
# Monkey-patch NLTK with better Tree display that works on Cloud or other display-less server.
print("Overriding nltk.tree.Tree pretty-printing to use custom GraphViz.")
treeviz.monkey_patch(nltk.tree.Tree, node_style_fn=None, format='svg')

# Test rendering
print("Sample tree to test rendering:")
nltk.tree.Tree.fromstring("(S (NP (PRP I)) (VP (VBP love) (NNP W266)) (SYM 😄))")

Overriding nltk.tree.Tree pretty-printing to use custom GraphViz.
Sample tree to test rendering:


Tree('S', [Tree('NP', [Tree('PRP', ['I'])]), Tree('VP', [Tree('VBP', ['love']), Tree('NNP', ['W266'])]), Tree('SYM', ['😄'])])

In [28]:
# Import some useful libraries...
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import os, sys, collections
import copy
from importlib import reload

import numpy as np
import nltk
from nltk.tree import Tree
from IPython.display import display, HTML
from tqdm import tqdm as ProgressBar

import logging
logging.basicConfig(stream=sys.stderr, level=logging.DEBUG)

# Helpers for this assignment
from w266_common import utils, treeviz
import part2_helpers
import pcfg, pcfg_test
import cky, cky_test

# Use the sample of the Penn Treebank included with NLTK.
assert(nltk.download('treebank'))
corpus = nltk.corpus.treebank

# If you do install the full Penn Treebank, change the "False" to "True" below.
use_full_ptb = False
if use_full_ptb:
    part2_helpers.verify_ptb_install()
    corpus = nltk.corpus.ptb  # Throws errors, for some reason
    # This configures the corpus to use the WSJ section only.
    # The Brown section has some mis-bracketed trees that will cause the 
    # corpus reader to throw (many) errors.
    if not hasattr(corpus, '_parsed_sents'):
        print("Monkey-patching corpus reader...")
        corpus._parsed_sents = corpus.parsed_sents
        corpus.parsed_sents = lambda: corpus._parsed_sents(categories=['news'])

[nltk_data] Downloading package treebank to
[nltk_data]     /home/yeunghoman/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


## Part (a) Exercises: API warm-up

We'll be using `nltk.tree.Tree` objects in the rest of the assignment, which provide some convenient methods for viewing and traversing parse trees, as well as extracting grammar rules (productions).

The API is documented here: http://www.nltk.org/api/nltk.html#nltk.tree.Tree

In the cells below, do the following to familiarize yourself with the Tree API:

1. Construct the tree `(S (NP foo) (VP bar))` using the Tree API. The constructor can be called as: `Tree(lhs, [rhs_1, rhs_2, ...]` where `lhs` is a string and `rhs_*` are either trees or strings.
2. Display the tree for the first sentence in the corpus. (It should be: "Pierre Vinken , 61 years old , will join the board as a nonexecutive director Nov. 29 .")
3. For the first sentence, print the label of the root node and the labels of its immediate children.
4. For the first sentence, print all the productions in the sentence. Also print the LHS of the first production, and the RHS of the second production.


In [4]:
## Part (a).1
#### YOUR CODE HERE ####
Tree('S', [Tree('NP',['foo']),Tree('VP',['bar'])])
#### END(YOUR CODE) ####

Tree('S', [Tree('NP', ['foo']), Tree('VP', ['bar'])])

In [11]:
## Part (a).2
#### YOUR CODE HERE ####
corpus.parsed_sents()[0]
#### END(YOUR CODE) ####

Tree('S', [Tree('NP-SBJ', [Tree('NP', [Tree('NNP', ['Pierre']), Tree('NNP', ['Vinken'])]), Tree(',', [',']), Tree('ADJP', [Tree('NP', [Tree('CD', ['61']), Tree('NNS', ['years'])]), Tree('JJ', ['old'])]), Tree(',', [','])]), Tree('VP', [Tree('MD', ['will']), Tree('VP', [Tree('VB', ['join']), Tree('NP', [Tree('DT', ['the']), Tree('NN', ['board'])]), Tree('PP-CLR', [Tree('IN', ['as']), Tree('NP', [Tree('DT', ['a']), Tree('JJ', ['nonexecutive']), Tree('NN', ['director'])])]), Tree('NP-TMP', [Tree('NNP', ['Nov.']), Tree('CD', ['29'])])])]), Tree('.', ['.'])])

(Side node: it turns out this first sentence is actually pretty meta: http://languagelog.ldc.upenn.edu/nll/?p=3594 Long live Pierre Vinken!)

In [3]:
## Part (a).3
#### YOUR CODE HERE ####
s = corpus.parsed_sents()[0]

print ("root label:", s.label())
for i in range(len(s)):
    print ("immediate children label:",s[i].label())
#### END(YOUR CODE) ####

root label: S
immediate children label: NP-SBJ
immediate children label: VP
immediate children label: .


In [53]:
## Part (a).4
#### YOUR CODE HERE ####
s = corpus.parsed_sents()[0]
print("All productions:")
print(s.productions())
print()
print("LHS first production:")
print(s.productions()[0].lhs())
print()
print("RHS second production")
print(s.productions()[1].rhs())
#### END(YOUR CODE) ####

All productions:
[S -> NP-SBJ VP ., NP-SBJ -> NP , ADJP ,, NP -> NNP NNP, NNP -> 'Pierre', NNP -> 'Vinken', , -> ',', ADJP -> NP JJ, NP -> CD NNS, CD -> '61', NNS -> 'years', JJ -> 'old', , -> ',', VP -> MD VP, MD -> 'will', VP -> VB NP PP-CLR NP-TMP, VB -> 'join', NP -> DT NN, DT -> 'the', NN -> 'board', PP-CLR -> IN NP, IN -> 'as', NP -> DT JJ NN, DT -> 'a', JJ -> 'nonexecutive', NN -> 'director', NP-TMP -> NNP CD, NNP -> 'Nov.', CD -> '29', . -> '.']

LHS first production:
S

RHS second production
(NP, ,, ADJP, ,)


# (a) Preprocessing

### Removing Cross-References

This first step of preprocessing takes the treebank, strips out the cross references (NPs are wrapped by special nodes that assign index numbers to them so that coreferences can be indicated).  Unfortunately, this also injects a NP-SBJ-# node between nodes you'd expect to produce one another.  Since the # changes throughout the corpus, our counts of the production rules all end up being 1 - and useless.

See NP-SBJ-1 in the tree below.  Note there is also a NP-SBJ leading to a NONE/1 subtree as a crossreference later.

In the code below we skip over nodes whose label start with NP-, connecting any children nodes to the NP-'s parent.  We also snip out any subtrees rooted by NONE.  The tree above is printed again after this next cell to illustrate the effect of this code.

### Chomsky Normal Form

Finally, CKY assumes that trees are constructed from a grammar that is in [Chomsky normal form](https://en.wikipedia.org/wiki/Chomsky_normal_form).

This means that the grammar only consists of three types of rules:
- **Binary nonterminal:** `A -> B C`
- **Unary preterminal:** `A -> a`
- **Epsilon:** `A -> `$\ \ \epsilon$

where `A`, `B`, and `C`, are non-terminals, `a` is a terminal, and $\epsilon$ is the empty sentence.

In order to accomplish this, we add new non-terminals to the language and build longer sequences of non-terminals through them.   
For example, the ternary rule
- `A -> B C D`

becomes two rules:
- `A -> B A|<C-D>`
- `A|<C-D> -> C D`

where `A|<C-D>` is a dummy symbol that we add to signifiy that it's a production of `A` that creates `C D`.

Since the resulting tree is (at most) binary, we sometimes call this process _binarization_.

#### Horizontal Markovization

The dummy-symbol system works well until you get very long grammar rules such as
- `A -> B C D E F G H I J K L`

If we followed the rule above, we'd get intermediate symbols that look like `A|<B-C-D-E-F-G-H-...>`. This would quickly lead to an explosion in the number of symbols in our grammar! Because such long productions are fairly rare, we may have trouble getting good estimates of their probability. (*Recall the sparsity problem from Week 2!*)

One way to counter this is called _horizontal Markovization_. Similar to how in language modeling, we "forgot" all history more than (n-1) words back, we can simply choose to truncate the the history and only store shorter symbols like `A|<B-C>`, `A|<C-D>`, `A|<D-E>`, and so on. This way, we can share parameters across more examples that are similar in structure.

NLTK implements this for us in the `chomsky_normal_form` function; try changing the **`horzMarkov`** parameter below to see how it works.

Take a minute to play with the ```horzMarkov``` parameter in the block below to see how this works. 

In [6]:
sentence = corpus.parsed_sents()[35]
sentence

Tree('S', [Tree('NP-SBJ', [Tree('NN', ['Compound']), Tree('NNS', ['yields'])]), Tree('VP', [Tree('VBP', ['assume']), Tree('UCP', [Tree('NP', [Tree('NP', [Tree('NN', ['reinvestment'])]), Tree('PP', [Tree('IN', ['of']), Tree('NP', [Tree('NNS', ['dividends'])])])]), Tree('CC', ['and']), Tree('SBAR', [Tree('IN', ['that']), Tree('S', [Tree('NP-SBJ', [Tree('DT', ['the']), Tree('JJ', ['current']), Tree('NN', ['yield'])]), Tree('VP', [Tree('VBZ', ['continues']), Tree('PP-TMP', [Tree('IN', ['for']), Tree('NP', [Tree('DT', ['a']), Tree('NN', ['year'])])])])])])])]), Tree('.', ['.'])])

In [7]:
# Filter out NP-* nodes.
cleaned_sentence = part2_helpers.clean_tree(sentence, simplify=True)
display(HTML(treeviz.render_tree(cleaned_sentence, title="Original", format='svg')))

In [8]:
# Convert sentence to Chomsky normal form.
cnf_sentence4 = copy.deepcopy(cleaned_sentence)
nltk.treetransforms.chomsky_normal_form(cnf_sentence4, horzMarkov=4)
display(HTML(treeviz.render_tree(cnf_sentence4, title="Binarized (CNF)", format='svg')))

In [9]:
# Convert sentence to Chomsky normal form.
cnf_sentence3 = copy.deepcopy(cleaned_sentence)
nltk.treetransforms.chomsky_normal_form(cnf_sentence3, horzMarkov=3)
display(HTML(treeviz.render_tree(cnf_sentence3, title="Binarized (CNF)", format='svg')))

In [10]:
# Convert sentence to Chomsky normal form.
cnf_sentence = copy.deepcopy(cleaned_sentence)
nltk.treetransforms.chomsky_normal_form(cnf_sentence, horzMarkov=2)
display(HTML(treeviz.render_tree(cnf_sentence, title="Binarized (CNF)", format='svg')))

In [16]:
# Convert sentence to Chomsky normal form.
cnf_sentence1 = copy.deepcopy(cleaned_sentence)
nltk.treetransforms.chomsky_normal_form(cnf_sentence1, horzMarkov= 1)
display(HTML(treeviz.render_tree(cnf_sentence1, title="Binarized (CNF)", format='svg')))

In [17]:
# Convert sentence to Chomsky normal form.
cnf_sentence0 = copy.deepcopy(cleaned_sentence)
nltk.treetransforms.chomsky_normal_form(cnf_sentence0, horzMarkov= 0)
display(HTML(treeviz.render_tree(cnf_sentence0, title="Binarized (CNF)", format='svg')))

### Run Pre-Processing on Corpus

We'll loop through the whole corpus, and make copies of each sentence in CNF form. Use the `cnf_sentences` list for training the grammar in part (b).

**Note:** if you're using the `treebank` corpus sample, this should run in just a few seconds. But if you use the full Penn Treebank, it'll take around 1-2 minutes to process all the trees. If you get an error "`AttributeError: 'tqdm' object has no attribute 'miniters'`", ignore it - the code should still work.

In [11]:
# Preprocess the treebank.
cleaned_sentences = []
cnf_sentences = []
for sentence in ProgressBar(corpus.parsed_sents(), desc="Processing sentences"):
    # Filter out NP-* nodes.
    cleaned_sentence = part2_helpers.clean_tree(sentence, simplify=True)
    cleaned_sentences.append(cleaned_sentence)
    
    # Convert sentence to Chomsky normal form.
    cnf_sentence = copy.deepcopy(cleaned_sentence)
    nltk.treetransforms.chomsky_normal_form(cnf_sentence, horzMarkov=2)
    cnf_sentences.append(cnf_sentence)

Processing sentences: 100%|██████████| 3914/3914 [00:06<00:00, 591.81it/s]


# (b) Production rule probabilities

In this next section, you'll compute about production rule probabilities.

We won't use epsilon rules, so all of our rules will be of the form:
- Binary nonterminal: `A -> B C`
- Unary preterminal: `A -> a`

The left hand side (LHS) of these rules only ever consist of a single nonterminal.  The right hand side (RHS) consists of either two non-terminals or one terminal.

We'll do this in two stages:
- Count LHS, and (LHS,RHS) each in their own dict
- Calculate $ P(RHS | LHS) = \frac{count(LHS, RHS)}{count(LHS)} $

## Part (b) Implementation: Training the grammar

Now that you're comfortable with NLTK Tree objects, let's use them to build our grammar. We've implemented a skeleton in **`pcfg.py`**; your job is to finish the implementation of the `pcfg.PCFG` class. Specifically:

- Implement `update_counts`, which updates the production counts for a single sentence.
- Implement `compute_scores`, which computes log-probabilities.

Read the documentation in `pcfg.py` for the names of the data structures you should populate, and their precise types. Both functions should be straightforward, and only require a couple of lines of code each!

#### Indexing Rules

Your code here need only deal with straightforward maps of productions, but in order to parse efficiently we need to build an inverted index, keyed on the rule's RHS. This way, we can quickly look up rules (and their scores) that would combine two subtrees during the CKY algorithm.

We've implemented this for you in `pcfg.PCFG.build_index()`, but you'll want to look carefully at how that function works - when you implement CKY in part(c), you'll make heavy use of the `grammar.parsing_index` structure.

### Testing `update_counts()`

If everything works, you should see this in the cell below:

```
Top productions:
(PP -> IN NP, 7369)
(, -> ',', 4885)
(DT -> 'the', 4038)
(. -> '.', 3828)
(S|<VP-.> -> VP ., 3071)
(NP -> NP PP, 2644)
(S -> VP, 2335)
(IN -> 'of', 2319)
(TO -> 'to', 2161)
(NP -> DT NN, 2020)
```

### Testing `compute_scores()`

If everything went well, you should see:
```
food [(NN, -6.7128043057880404)]
a [(DT, -1.4717815426061982), (JJ, -7.9783109698677208), (IN, -9.1959371416654392), (LS, -2.5649493574615367)]
I [(NNP, -8.4563810520194806), (PRP, -2.7203634613355669)]
```

In [12]:
reload(pcfg)
utils.run_tests(pcfg_test, ["TestPCFG"])

test_pcfg (pcfg_test.TestPCFG) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


In [13]:
reload(pcfg)

grammar = pcfg.PCFG()
for sentence in ProgressBar(cnf_sentences, desc="Counting productions"):
    grammar.update_counts(sentence)
    
print("Top productions:")
for p in grammar.top_productions():  # Top productions, by un-normalized count
    print(p)
print("")

grammar.compute_scores()  # compute log-probabilities
grammar.build_index()     # prepare for parsing

for w in ['food', 'a', 'I']:
    print(w, grammar.parsing_index[(w,)])

Counting productions: 100%|██████████| 3914/3914 [00:02<00:00, 1679.45it/s]


Top productions:
(PP -> IN NP, 7369)
(, -> ',', 4885)
(DT -> 'the', 4038)
(. -> '.', 3828)
(S|<VP-.> -> VP ., 3071)
(NP -> NP PP, 2644)
(S -> VP, 2335)
(IN -> 'of', 2319)
(TO -> 'to', 2161)
(NP -> DT NN, 2020)

food [(NN, -6.7128043057880404)]
a [(DT, -1.4717815426061982), (JJ, -7.9783109698677208), (IN, -9.1959371416654392), (LS, -2.5649493574615367)]
I [(NNP, -8.4563810520194806), (PRP, -2.7203634613355669)]


You don't need to do anything with this next cell except to run it.

It's not particularly useful, but if you need to keep track of what each variable contains, this provides a useful reference.

In [14]:
print('Productions (nltk.grammar.Production):')
for (production, count) in grammar.production_counts.most_common(5):
    print (production, count)

print('\n\nLHS counts:')
for (lhs, count) in grammar.lhs_counts.most_common(5):
    print((lhs, count))
    
print('\n\nLog Probabilities:')
print('\n'.join([str(x) for x in grammar.parsing_index.items()][0:10]))

Productions (nltk.grammar.Production):
PP -> IN NP 7369
, -> ',' 4885
DT -> 'the' 4038
. -> '.' 3828
S|<VP-.> -> VP . 3071


LHS counts:
(NP, 23766)
(VP, 14524)
(NN, 13166)
(S, 9946)
(IN, 9857)


Log Probabilities:
((NP, S|<,-ADJP>), [(S, -7.413166270046629), (S|<NP-,>, -4.8978397999509111)])
((NNP, NNP), [(NP, -3.4761407676371343), (NP|<NNP-NNP>, -0.5729867009612386), (PP|<NNP-NNP>, -0.68056839835308525), (NX, -3.5624655292582776), (NX|<NNP-NNP>, 0.0), (S, -8.5117785587147381), (VP|<NNP-NNP>, -1.4663370687934272), (SINV|<NNP-NNP>, -2.4849066497879999), (ADJP, -6.0768774249561117), (UCP|<NNP-NNP>, 0.0)])
(('Pierre',), [(NNP, -9.149528232579426)])
(('Vinken',), [(NNP, -8.4563810520194806)])
((,, S|<ADJP-,>), [(S|<,-ADJP>, -0.095310179804324768)])
((',',), [(,, -0.00020468734080303363)])
((ADJP, S|<,-VP>), [(S|<ADJP-,>, -0.095310179804324768)])
((NP, JJ), [(ADJP, -4.7775944408258511), (ADVP, -6.9608219999188288), (NP, -10.076011266849971)])
((CD, NNS), [(NP, -4.5114908595272771), (QP|<CD

# (c) Implement CKY

After that bit of preamble, you only have one more task to go!  It's a big one though, so do take your time and get things right. 

**In `cky.py`, implement the `CKY_apply_preterminal_rules` and `CKY_apply_binary_rules` functions.**

We've set up the skeleton of the CKY algorithm for you in the `CKY` function; be sure to read the in-line comments there carefully before you start. The outline is:
1. Construct the chart (`make_chart`, provided)
2. Populate the first row of the chart using preterminal rules (`CKY_apply_preterminal_rules`)
3. Populate the rest of the chart using binary rules (`CKY_apply_binary_rules`)
4. Read off the top cell for the final derivation. (provided)

We'll implement the chart itself as a dict that you can index into first by cell position and then by non-terminal like this:
```
chart[(0, 1)][NN]
```

The value is an [nltk.tree.ProbabilisticTree](http://www.nltk.org/api/nltk.html#nltk.tree.ProbabilisticTree), which is just like an `nltk.tree.Tree` except that it has an additional `logprob()` method that returns the score (log-probability). Similarly, the constructor takes an additional argument: `pt = ProbabilisticTree(lhs, (rhs_1,...), logprob=score)` - you should use this to construct the backtrace trees.

See the in-line comments in `cky.py` for additional hints and advice.

In [15]:
reload(cky)
utils.run_tests(cky_test, ["TestParsing"])

test_failing_rule_application (cky_test.TestParsing) ... ok
test_rule_application (cky_test.TestParsing) ... ok
test_rule_application_three_words (cky_test.TestParsing) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.009s

OK


In [16]:
reload(cky)
parser = cky.CKYParser(grammar)
derivation = parser.parse('I eat red hot food with a knife'.split(), 'S')

assert round(derivation.logprob(), 2) == -64.08
derivation

ProbabilisticTree(S, [ProbabilisticTree(PRP, ['I']) (p=0.15173612851696769), ProbabilisticTree(VP, [ProbabilisticTree(VBP, ['eat']) (p=0.0068668096876260687), ProbabilisticTree(VP|<NP-PP>, [ProbabilisticTree(NP, [ProbabilisticTree(JJ, ['red']) (p=0.0039654190017090543), ProbabilisticTree(NP|<JJ-NN>, [ProbabilisticTree(JJ, ['hot']) (p=0.0039654190017090543), ProbabilisticTree(NN, ['food']) (p=0.0095333282960814777)]) (p=3.2078091828121638e-05)]) (p=1.9662155367311963e-09), ProbabilisticTree(PP, [ProbabilisticTree(IN, ['with']) (p=0.10602582040574654), ProbabilisticTree(NP, [ProbabilisticTree(DT, ['a']) (p=0.36053680734251442), ProbabilisticTree(NN, ['knife']) (p=0.0013951219699423914)]) (p=9.1090931043064175e-05)]) (p=8.1464414595723497e-06)]) (p=1.3501972707728444e-14)]) (p=1.8692043100608051e-18)]) (p=5.1222272930921724e-20)

### Experimentation

Try a few more sentences.  Do you notice any patterns with your results?  Any common types of errors?  Are these an artifact of CKY, or of how you did the markovization/counting?

Put any code you write in the next cell and a writeup of the results in the cell after.

(If you have a format that's more natural to your description, feel free to deviate from this format.)

In [19]:
#### YOUR CODE HERE ####
parser.parse('I eat some food'.split(), 'S')
#### END(YOUR CODE) ####

ProbabilisticTree(S, [ProbabilisticTree(PRP, ['I']) (p=0.15173612851696769), ProbabilisticTree(VP, [ProbabilisticTree(VBP, ['eat']) (p=0.0068668096876260687), ProbabilisticTree(NP, [ProbabilisticTree(DT, ['some']) (p=0.054274150092784218), ProbabilisticTree(NN, ['food']) (p=0.0095333282960814777)]) (p=9.3702447504789157e-05)]) (p=3.1264486113032746e-08)]) (p=8.5674852775975288e-10)

In [38]:
#### YOUR CODE HERE ####
parser.parse('I ran away'.split(), 'S')
#### END(YOUR CODE) ####

ProbabilisticTree(S, [ProbabilisticTree(PRP, ['I']) (p=0.15173612851696769), ProbabilisticTree(VP, [ProbabilisticTree(VBD, ['ran']) (p=0.013333139344354423), ProbabilisticTree(RB, ['away']) (p=0.024009484276474671)]) (p=4.1723132474157541e-07)]) (p=1.143349428214613e-08)

#### Observation 1 : Error - OOV words

With a larger dataset such as the full Penn Tree Bank (note: the code failed to load with `use_full_ptb=True`) this problem can be alleviated. But in general, the given implementation of CKY is always going to throw an error for OOV words. We can probably solve this by discounting probabilities from seen words in training set to unseen `<unk>` words in the test set. A different markovization setting will not help because only one-to-one preterminal-terminal probabilities matter.

In [20]:
#### YOUR CODE HERE ####
parser.parse('I eat the apple with a spoon'.split(), 'S')
#### END(YOUR CODE) ####

In [39]:
#### YOUR CODE HERE ####
parser.parse('The horse raced past the barn fell'.split(), 'S')
#### END(YOUR CODE) ####

In [40]:
#### YOUR CODE HERE ####
parser.parse('The old man the boat'.split(), 'S')
#### END(YOUR CODE) ####

In [43]:
#### YOUR CODE HERE ####
parser.parse('Fat people eat accumulates'.split(), 'S')
#### END(YOUR CODE) ####

In [46]:
#### YOUR CODE HERE ####
parser.parse('The girl told the story cried'.split(), 'S')
#### END(YOUR CODE) ####

#### Observation 2: Garden Path Sentences

The parser fails to parse both garden path sentences below. In both `the old man the house` and `the prime number few`, the parser fails to recognize that `man` and `number` can be `verbs`. Therefore, the parser at best stitched `NP` and other `NN` or `PP` together. It fails to see any `VP`s. This is a limitation of what the parser has seen when its grammar was built.

Notice that with a different horizontal markovization = 3, the parser produced a different parse and lower score for `the old man the house`. Mostly because the production rules and probabilities it stores when its grammar was built is a little different. The parser correctly recognize that `the old` is the `NP`, correctly put `the` and `house` under a `NP`, but still fails to see that `man the house` should be a `VP`.

In [92]:
markovization = 2  # tune between 2 to 4

# Preprocess the treebank.
cleaned_sentences = []
cnf_sentences = []
for sentence in ProgressBar(corpus.parsed_sents(), desc="Processing sentences"):
    # Filter out NP-* nodes.
    cleaned_sentence = part2_helpers.clean_tree(sentence, simplify=True)
    cleaned_sentences.append(cleaned_sentence)
    
    # Convert sentence to Chomsky normal form.
    cnf_sentence = copy.deepcopy(cleaned_sentence)
    nltk.treetransforms.chomsky_normal_form(cnf_sentence, horzMarkov=markovization)
    cnf_sentences.append(cnf_sentence)
    

reload(pcfg)
grammar = pcfg.PCFG()
for sentence in ProgressBar(cnf_sentences, desc="Counting productions"):
    grammar.update_counts(sentence)
grammar.compute_scores()  # compute log-probabilities
grammar.build_index()     # prepare for parsing


reload(cky)
parser = cky.CKYParser(grammar)

Counting productions: 100%|██████████| 3914/3914 [00:02<00:00, 1834.68it/s]


In [32]:
#### YOUR CODE HERE ####
print ("markovization = 2")
parser.parse('The old man the house'.split(), 'S')
#### END(YOUR CODE) ####

markovization = 2


ProbabilisticTree(S, [ProbabilisticTree(NP, [ProbabilisticTree(DT, ['The']) (p=0.18452259796200876), ProbabilisticTree(NP|<JJ-NN>, [ProbabilisticTree(JJ, ['old']) (p=0.022198341143999673), ProbabilisticTree(NN, ['man']) (p=0.0078098666024756982)]) (p=0.0001471089281806979)]) (p=2.8457255607423059e-06), ProbabilisticTree(S|<DT-NN>, [ProbabilisticTree(DT, ['the']) (p=0.61382222719119828), ProbabilisticTree(NN, ['house']) (p=0.0042569766272000193)]) (p=0.00011824194767351253)]) (p=9.2185778359287842e-13)

In [35]:
#### YOUR CODE HERE ####
print ("markovization = 3")
parser.parse('The old man the house'.split(), 'S')
#### END(YOUR CODE) ####

markovization = 3


ProbabilisticTree(S, [ProbabilisticTree(NP, [ProbabilisticTree(DT, ['The']) (p=0.18452259796200876), ProbabilisticTree(JJ, ['old']) (p=0.022198341143999673)]) (p=3.2335899789923262e-05), ProbabilisticTree(PP, [ProbabilisticTree(NN, ['man']) (p=0.0078098666024756982), ProbabilisticTree(NP, [ProbabilisticTree(DT, ['the']) (p=0.61382222719119828), ProbabilisticTree(NN, ['house']) (p=0.0042569766272000193)]) (p=0.00047321361447314977)]) (p=6.5027741124364119e-09)]) (p=1.9946022336376937e-15)

In [33]:
#### YOUR CODE HERE ####
print ("markovization = 2")
parser.parse('The prime number few'.split(), 'S')
#### END(YOUR CODE) ####

markovization = 2


ProbabilisticTree(S, [ProbabilisticTree(NNP, ['The']) (p=0.0046029233336108066), ProbabilisticTree(PP, [ProbabilisticTree(JJ, ['prime']) (p=0.0052522603199820599), ProbabilisticTree(ADJP, [ProbabilisticTree(NN, ['number']) (p=0.019522335439188361), ProbabilisticTree(JJ, ['few']) (p=0.027681183011934691)]) (p=3.7383398473712775e-06)]) (p=3.4547994704225997e-11)]) (p=4.3566826190029434e-16)

#### Observation 3: Failure to construct NP

In many sentences, the parser doesn't recognize that `the` and some `NN` can form a `NP`. This is a rather strange deficiency which a different horizontal markovization hasn't resolved. It is possible that during binarization, the grammar object stored more `dummy -> NN VP` then necessary.

In [55]:
#### YOUR CODE HERE ####
parser.parse('the man do the work'.split(), 'S')
#### END(YOUR CODE) ####

ProbabilisticTree(S, [ProbabilisticTree(DT, ['the']) (p=0.61382222719119828), ProbabilisticTree(S|<NN-VP>, [ProbabilisticTree(NN, ['man']) (p=0.0078098666024756982), ProbabilisticTree(VP, [ProbabilisticTree(VB, ['do']) (p=0.05009965531734592), ProbabilisticTree(NP, [ProbabilisticTree(DT, ['the']) (p=0.61382222719119828), ProbabilisticTree(NN, ['work']) (p=0.010739316235619989)]) (p=0.0011938028083960934)]) (p=8.0532876801950436e-06)]) (p=3.9456938837177686e-08)]) (p=2.669596637985588e-09)

In [84]:
#### YOUR CODE HERE ####
print ("markovization = 0")
parser.parse('the man do the work'.split(), 'S')
#### END(YOUR CODE) ####

markovization = 0


ProbabilisticTree(S, [ProbabilisticTree(DT, ['the']) (p=0.61382222719119828), ProbabilisticTree(S|<>, [ProbabilisticTree(NN, ['man']) (p=0.0078098666024756982), ProbabilisticTree(VP, [ProbabilisticTree(VB, ['do']) (p=0.05009965531734592), ProbabilisticTree(NP, [ProbabilisticTree(DT, ['the']) (p=0.61382222719119828), ProbabilisticTree(NN, ['work']) (p=0.010739316235619989)]) (p=0.0011938028083960934)]) (p=8.0532876801950436e-06)]) (p=6.7312664093996608e-09)]) (p=9.5556023728302394e-10)

In [51]:
#### YOUR CODE HERE ####
parser.parse('The man eat the food'.split(), 'S')
#### END(YOUR CODE) ####

ProbabilisticTree(S, [ProbabilisticTree(DT, ['The']) (p=0.18452259796200876), ProbabilisticTree(S|<NN-VP>, [ProbabilisticTree(NN, ['man']) (p=0.0078098666024756982), ProbabilisticTree(VP, [ProbabilisticTree(VBP, ['eat']) (p=0.0068668096876260687), ProbabilisticTree(NP, [ProbabilisticTree(DT, ['the']) (p=0.61382222719119828), ProbabilisticTree(NN, ['food']) (p=0.0095333282960814777)]) (p=0.0010597428964309659)]) (p=3.5359073269839226e-07)]) (p=1.7324114656655868e-09)]) (p=3.5235485293482851e-11)

In [52]:
#### YOUR CODE HERE ####
parser.parse('The girl eat the food'.split(), 'S')
#### END(YOUR CODE) ####

ProbabilisticTree(S, [ProbabilisticTree(DT, ['The']) (p=0.18452259796200876), ProbabilisticTree(S|<NN-VP>, [ProbabilisticTree(NN, ['girl']) (p=0.0022556425095938077), ProbabilisticTree(VP, [ProbabilisticTree(VBP, ['eat']) (p=0.0068668096876260687), ProbabilisticTree(NP, [ProbabilisticTree(DT, ['the']) (p=0.61382222719119828), ProbabilisticTree(NN, ['food']) (p=0.0095333282960814777)]) (p=0.0010597428964309659)]) (p=3.5359073269839226e-07)]) (p=5.0035437798954011e-10)]) (p=1.0176698594179949e-11)

#### Observation 4: Failure to parse

Despite some word sense error (no `ate` available in the grammar), the following sentence should parse. But because no smoothing was implemented in the CKY algorithm, the limited Penn Tree Bank sample data didn't train the parser to recognize enough common production rules that can be grammatically correct. A different horizontal markovization did not help.

In [75]:
#### YOUR CODE HERE ####
print ("markovization = 2")
parser.parse('The girl who eat the food dies'.split(), 'S')
#### END(YOUR CODE) ####

markovization = 2


In [78]:
#### YOUR CODE HERE ####
print ("markovization = 3")
parser.parse('The girl who eat the food died'.split(), 'S')
#### END(YOUR CODE) ####

markovization = 3


#### Observation 5: Attachment Ambiguity

The following two sentences should have different parses. In `I eat the food with sugar`, although this is bad english, a good parser should put `food with sugar` as one `NP`. Since grammar is built statistically without any semantics information, the parsing algorithm cannot pick up such semantics.

In [67]:
#### YOUR CODE HERE ####
print ("markovization = 1")
parser.parse('I eat food with knife'.split(), 'S')
#### END(YOUR CODE) ####

markovization = 1


ProbabilisticTree(S, [ProbabilisticTree(PRP, ['I']) (p=0.15173612851696769), ProbabilisticTree(VP, [ProbabilisticTree(VBP, ['eat']) (p=0.0068668096876260687), ProbabilisticTree(VP|<NN>, [ProbabilisticTree(NN, ['food']) (p=0.0095333282960814777), ProbabilisticTree(PP, [ProbabilisticTree(IN, ['with']) (p=0.10602582040574654), ProbabilisticTree(NN, ['knife']) (p=0.0013951219699423914)]) (p=4.2080387811938611e-07)]) (p=1.7756276908385103e-09)]) (p=4.8490612014471572e-14)]) (p=1.3288003616426014e-15)

In [63]:
#### YOUR CODE HERE ####
print ("markovization = 2")
parser.parse('I eat food with knife'.split(), 'S')
#### END(YOUR CODE) ####

ProbabilisticTree(S, [ProbabilisticTree(PRP, ['I']) (p=0.15173612851696769), ProbabilisticTree(VP, [ProbabilisticTree(VBP, ['eat']) (p=0.0068668096876260687), ProbabilisticTree(VP|<NN-PP>, [ProbabilisticTree(NN, ['food']) (p=0.0095333282960814777), ProbabilisticTree(PP, [ProbabilisticTree(IN, ['with']) (p=0.10602582040574654), ProbabilisticTree(NN, ['knife']) (p=0.0013951219699423914)]) (p=4.2080387811938611e-07)]) (p=2.8932507520941623e-09)]) (p=6.7689129455197821e-14)]) (p=1.8549021338913268e-15)

In [68]:
#### YOUR CODE HERE ####
print ("markovization = 1")
parser.parse('I eat food with sugar'.split(), 'S')
#### END(YOUR CODE) ####

markovization = 1


ProbabilisticTree(S, [ProbabilisticTree(PRP, ['I']) (p=0.15173612851696769), ProbabilisticTree(VP, [ProbabilisticTree(VBP, ['eat']) (p=0.0068668096876260687), ProbabilisticTree(VP|<NN>, [ProbabilisticTree(NN, ['food']) (p=0.0095333282960814777), ProbabilisticTree(PP, [ProbabilisticTree(IN, ['with']) (p=0.10602582040574654), ProbabilisticTree(NN, ['sugar']) (p=0.012259986391737726)]) (p=3.6979202754201958e-06)]) (p=1.5603776440687641e-08)]) (p=4.2612349044220394e-13)]) (p=1.1677168521507241e-14)

In [65]:
#### YOUR CODE HERE ####
print ("markovization = 2")
parser.parse('I eat food with sugar'.split(), 'S')
#### END(YOUR CODE) ####

ProbabilisticTree(S, [ProbabilisticTree(PRP, ['I']) (p=0.15173612851696769), ProbabilisticTree(VP, [ProbabilisticTree(VBP, ['eat']) (p=0.0068668096876260687), ProbabilisticTree(VP|<NN-PP>, [ProbabilisticTree(NN, ['food']) (p=0.0095333282960814777), ProbabilisticTree(PP, [ProbabilisticTree(IN, ['with']) (p=0.10602582040574654), ProbabilisticTree(NN, ['sugar']) (p=0.012259986391737726)]) (p=3.6979202754201958e-06)]) (p=2.5425171141146952e-08)]) (p=5.9483530749900399e-13)]) (p=1.630042061516097e-14)

#### Observation 6: Coordination Ambiguity

The parser correctly resolve that `man and woman` are both `old` in the first sentence, but incorrectly resolve `man and girl` the same way. Only when `young girl` is explicitly used did the parser correctly parsed the second sentence. This is probably irrelevant to horizontal markovization but relevant to how most examples the parser had seen use a `JJ` to modify `NP`s that are constructed out of `NN`, `CC` and `NN`. The algorithm doesn't have the ability to attach `JJ` to only a single `NN` before a word like `and` because statistically it is a rare case.

In [86]:
#### YOUR CODE HERE ####
print ("markovization = 2")
parser.parse('the old man and woman'.split(), 'S')
#### END(YOUR CODE) ####

markovization = 2


ProbabilisticTree(S, [ProbabilisticTree(DT, ['the']) (p=0.61382222719119828), ProbabilisticTree(VP, [ProbabilisticTree(JJ, ['old']) (p=0.022198341143999673), ProbabilisticTree(NP, [ProbabilisticTree(NN, ['man']) (p=0.0078098666024756982), ProbabilisticTree(NP|<CC-NN>, [ProbabilisticTree(CC, ['and']) (p=0.75325911945695467), ProbabilisticTree(NN, ['woman']) (p=0.0063979771234287603)]) (p=0.0031713418091951816)]) (p=2.7507538629840954e-07)]) (p=2.7555433991878703e-11)]) (p=3.2347801305095663e-13)

In [87]:
#### YOUR CODE HERE ####
print ("markovization = 2")
parser.parse('the old man and girl'.split(), 'S')
#### END(YOUR CODE) ####

markovization = 2


ProbabilisticTree(S, [ProbabilisticTree(DT, ['the']) (p=0.61382222719119828), ProbabilisticTree(VP, [ProbabilisticTree(JJ, ['old']) (p=0.022198341143999673), ProbabilisticTree(NP, [ProbabilisticTree(NN, ['man']) (p=0.0078098666024756982), ProbabilisticTree(NP|<CC-NN>, [ProbabilisticTree(CC, ['and']) (p=0.75325911945695467), ProbabilisticTree(NN, ['girl']) (p=0.0022556425095938077)]) (p=0.00111807423803966)]) (p=9.6979361243028599e-08)]) (p=9.7148219012508759e-12)]) (p=1.1404397719472444e-13)

In [89]:
#### YOUR CODE HERE ####
print ("markovization = 2")
parser.parse('the old man and young girl'.split(), 'S')
#### END(YOUR CODE) ####

markovization = 2


ProbabilisticTree(S, [ProbabilisticTree(NP, [ProbabilisticTree(DT, ['the']) (p=0.61382222719119828), ProbabilisticTree(NP|<JJ-NN>, [ProbabilisticTree(JJ, ['old']) (p=0.022198341143999673), ProbabilisticTree(NN, ['man']) (p=0.0078098666024756982)]) (p=0.0001471089281806979)]) (p=9.4664264483713908e-06), ProbabilisticTree(S|<CC-NP>, [ProbabilisticTree(CC, ['and']) (p=0.75325911945695467), ProbabilisticTree(NP, [ProbabilisticTree(JJ, ['young']) (p=0.013729743651553623), ProbabilisticTree(NN, ['girl']) (p=0.0022556425095938077)]) (p=1.7936603103336015e-06)]) (p=9.5816140106757697e-08)]) (p=2.5336445496376694e-14)